In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from glob import glob
import warnings
from scipy.fft import fft, fftfreq
import matplotlib.pyplot as plt
from tqdm import tqdm
import json

warnings.filterwarnings('ignore')

In [ ]:
def calculate_rms_error(healthy_signal,damaged_signal):
    """Calculate RMS error between healthy and damaged signals"""
    return np.sqrt(np.mean((health_signal-damaged_signal)**2))

In [ ]:
def extract_dominant_frequency(signal_data,sampling_data=100):
    """Extract Dominant Frequency from signal using FFT"""
    n= len(signal_data)
    yf = fft(signal_data)
    xf = fftreq(n,1/sampling_rate)

    positive_freq_idx = xf>0
    xf_pos = xf[positive_freq_idx]
    yf_pos = np.abs(yf[positive_freq_idx])

    dominant_idx = np.argmax(yf_pos)
    return xf_pos[dominant_idx]

In [ ]:
def calculate_frequency_shift(health_signal,damaged_signal,sampling_rate=100):
    """Calculate frequency shift between healthy and damaged signals"""
    healthy_freq = extract_dominant_frequency(healthy_signal,sampling_rate)
    damaged_freq = extract_dominant_frequency(damaged_signal,sampling_rate)
    return abs(damaged_freq-healthy_freq)

# GPU-ACCELERATED PSO IMPLEMENTATION

In [ ]:
class BridgePSO:
    """GPU-accelerated Particle Swarm Optimization for damage localization"""
    
    def __init__(self, n_particles=50, n_elements=3, bounds=(0.1, 1.0), 
                 c1=0.5, c2=0.3, w=0.9, device='cuda'):
        """
        Initialize PSO optimizer
        
        Args:
            n_particles: Number of particles in swarm
            n_elements: Number of structural elements (sensors)
            bounds: Tuple of (lower, upper) bounds for stiffness reduction
            c1: Cognitive parameter
            c2: Social parameter
            w: Inertia weight
            device: 'cuda' or 'cpu'
        """
        self.n_particles = n_particles
        self.n_elements = n_elements
        self.device = torch.device(device if torch.cuda.is_available() else 'cpu')
        
        # PSO parameters
        self.c1 = c1
        self.c2 = c2
        self.w = w
        self.bounds = bounds
        
        # Initialize particles (stiffness reduction factors for each element)
        # 1.0 = healthy, < 1.0 = damaged
        self.positions = torch.rand(n_particles, n_elements, device=self.device)
        self.positions = self.positions * (bounds[1] - bounds[0]) + bounds[0]
        
        # Initialize velocities
        self.velocities = torch.randn(n_particles, n_elements, device=self.device) * 0.1
        
        # Best positions
        self.personal_best_positions = self.positions.clone()
        self.personal_best_scores = torch.full((n_particles,), float('inf'), device=self.device)
        
        self.global_best_position = None
        self.global_best_score = float('inf')
        
        # History tracking
        self.history = {
            'global_best_scores': [],
            'mean_scores': [],
            'std_scores': []
        }
    
    def apply_damage_to_signals(self, healthy_signals, damage_factors):
        """
        Apply damage factors to healthy signals to simulate damaged response
        
        Args:
            healthy_signals: Tensor of shape (n_sensors, signal_length)
            damage_factors: Tensor of shape (n_sensors,) with values in [0, 1]
        
        Returns:
            Simulated damaged signals
        """
        # Simple damage model: reduce amplitude and shift frequency
        # This is a simplified physical model
        damaged = healthy_signals.clone()
        
        for i, factor in enumerate(damage_factors):
            # Amplitude reduction
            damaged[i] *= factor
            
            # Add noise proportional to damage severity
            noise_level = (1 - factor) * 0.1
            damaged[i] += torch.randn_like(damaged[i]) * noise_level * damaged[i].std()
        
        return damaged
    
    def fitness_function(self, particles, healthy_signals, observed_signals):
        """
        Evaluate fitness for all particles (GPU-accelerated)
        
        Args:
            particles: Tensor of shape (n_particles, n_elements)
            healthy_signals: Tensor of shape (n_sensors, signal_length)
            observed_signals: Tensor of shape (n_sensors, signal_length)
        
        Returns:
            Fitness scores for each particle (lower is better)
        """
        batch_size = particles.shape[0]
        n_sensors = healthy_signals.shape[0]
        
        fitness_scores = torch.zeros(batch_size, device=self.device)
        
        for i in range(batch_size):
            # Simulate damaged response with current particle's damage factors
            simulated = self.apply_damage_to_signals(healthy_signals, particles[i])
            
            # Calculate RMS error between simulated and observed
            error = torch.sqrt(torch.mean((simulated - observed_signals) ** 2))
            
            # Add regularization to prevent trivial solutions (all damaged)
            # Encourage sparsity in damage
            sparsity_penalty = torch.sum(1.0 - particles[i]) * 0.01
            
            fitness_scores[i] = error + sparsity_penalty
        
        return fitness_scores
    
    def update(self, fitness_scores):
        """Update particle positions and velocities"""
        # Update personal bests
        improved = fitness_scores < self.personal_best_scores
        self.personal_best_scores[improved] = fitness_scores[improved]
        self.personal_best_positions[improved] = self.positions[improved].clone()
        
        # Update global best
        min_idx = torch.argmin(fitness_scores)
        if fitness_scores[min_idx] < self.global_best_score:
            self.global_best_score = fitness_scores[min_idx].item()
            self.global_best_position = self.positions[min_idx].clone()
        
        # Update velocities
        r1 = torch.rand_like(self.positions)
        r2 = torch.rand_like(self.positions)
        
        cognitive = self.c1 * r1 * (self.personal_best_positions - self.positions)
        social = self.c2 * r2 * (self.global_best_position - self.positions)
        
        self.velocities = self.w * self.velocities + cognitive + social
        
        # Update positions
        self.positions = self.positions + self.velocities
        
        # Apply bounds
        self.positions = torch.clamp(self.positions, self.bounds[0], self.bounds[1])
    
    def optimize(self, healthy_signals, observed_signals, n_iterations=50, verbose=True):
        """
        Run PSO optimization
        
        Args:
            healthy_signals: Tensor of healthy sensor signals
            observed_signals: Tensor of observed (damaged) sensor signals
            n_iterations: Number of PSO iterations
            verbose: Print progress
        
        Returns:
            Best damage factors found
        """
        iterator = tqdm(range(n_iterations)) if verbose else range(n_iterations)
        
        for iteration in iterator:
            # Evaluate fitness
            fitness_scores = self.fitness_function(
                self.positions, 
                healthy_signals, 
                observed_signals
            )
            
            # Update particles
            self.update(fitness_scores)
            
            # Track history
            self.history['global_best_scores'].append(self.global_best_score)
            self.history['mean_scores'].append(fitness_scores.mean().item())
            self.history['std_scores'].append(fitness_scores.std().item())
            
            if verbose and iteration % 10 == 0:
                iterator.set_description(
                    f"Best Fitness: {self.global_best_score:.6f}"
                )
        
        return self.global_best_position


# DATA LOADING AND PROCESSING


In [ ]:
def load_sensor_data(csv_path, sensor_names=['Sensor_24', 'Sensor_31', 'Sensor_32']):
    """Load sensor data from CSV file"""
    df = pd.read_csv(csv_path)
    
    signals = []
    for sensor in sensor_names:
        # Get all columns for this sensor
        sensor_cols = [col for col in df.columns if sensor in col]
        if len(sensor_cols) > 0:
            # Take first acceleration column
            signal = df[sensor_cols[0]].values
            signals.append(signal)
    
    return np.array(signals)

def prepare_signals_for_pso(signals, max_length=1000):
    """Convert numpy signals to PyTorch tensors"""
    # Truncate or pad to consistent length
    processed = []
    for signal in signals:
        if len(signal) > max_length:
            signal = signal[:max_length]
        elif len(signal) < max_length:
            signal = np.pad(signal, (0, max_length - len(signal)), mode='constant')
        processed.append(signal)
    
    return torch.tensor(np.array(processed), dtype=torch.float32)

# EVALUATION AND VISUALIZATION

In [ ]:
def evaluate_pso_results(predicted_damage, true_damage, threshold=0.9):
    """
    Evaluate PSO damage localization results
    
    Args:
        predicted_damage: Array of predicted stiffness factors
        true_damage: Array of true stiffness factors (ground truth)
        threshold: Values above this are considered healthy
    
    Returns:
        Dictionary with evaluation metrics
    """
    pred_damaged = predicted_damage < threshold
    true_damaged = true_damage < threshold
    
    # Calculate metrics
    tp = np.sum(pred_damaged & true_damaged)
    fp = np.sum(pred_damaged & ~true_damaged)
    tn = np.sum(~pred_damaged & ~true_damaged)
    fn = np.sum(~pred_damaged & true_damaged)
    
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'true_positives': tp,
        'false_positives': fp,
        'true_negatives': tn,
        'false_negatives': fn
    }

def plot_pso_results(history, predicted_damage, sensor_names, save_path=None):
    """Plot PSO optimization results"""
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Plot 1: Fitness convergence
    ax = axes[0]
    ax.plot(history['global_best_scores'], label='Best Fitness', linewidth=2)
    ax.plot(history['mean_scores'], label='Mean Fitness', alpha=0.7)
    ax.fill_between(
        range(len(history['mean_scores'])),
        np.array(history['mean_scores']) - np.array(history['std_scores']),
        np.array(history['mean_scores']) + np.array(history['std_scores']),
        alpha=0.2
    )
    ax.set_xlabel('Iteration')
    ax.set_ylabel('Fitness (RMS Error)')
    ax.set_title('PSO Convergence')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # Plot 2: Damage localization
    ax = axes[1]
    damage_severity = 1.0 - predicted_damage
    colors = ['red' if d > 0.1 else 'green' for d in damage_severity]
    bars = ax.bar(sensor_names, damage_severity, color=colors, alpha=0.7, edgecolor='black')
    ax.axhline(y=0.1, color='orange', linestyle='--', label='Damage Threshold')
    ax.set_ylabel('Damage Severity (1 - Stiffness)')
    ax.set_title('Predicted Damage Localization')
    ax.legend()
    ax.grid(True, alpha=0.3, axis='y')
    
    # Add value labels on bars
    for bar, val in zip(bars, damage_severity):
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{val:.3f}', ha='center', va='bottom')
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.show()

# MAIN PIPELINE

In [ ]:
def run_pso_on_dataset(healthy_path, damage_paths, n_particles=50, n_iterations=50, 
                       max_samples=10, device='cuda'):
    """
    Run PSO on complete dataset
    
    Args:
        healthy_path: Path to healthy signal CSV
        damage_paths: List of paths to damaged signal CSVs
        n_particles: Number of PSO particles
        n_iterations: Number of PSO iterations
        max_samples: Maximum damage cases to process
        device: 'cuda' or 'cpu'
    """
    device = torch.device(device if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")
    
    sensor_names = ['Sensor_24', 'Sensor_31', 'Sensor_32']
    
    # Load healthy signals
    print("Loading healthy signals...")
    healthy_signals_np = load_sensor_data(healthy_path, sensor_names)
    healthy_signals = prepare_signals_for_pso(healthy_signals_np).to(device)
    
    print(f"Healthy signals shape: {healthy_signals.shape}")
    
    results = []
    
    # Process damage cases
    print(f"\nProcessing up to {max_samples} damage cases...")
    for idx, damage_path in enumerate(damage_paths[:max_samples]):
        print(f"\n{'='*60}")
        print(f"Processing: {damage_path.split('/')[-2]}")
        print(f"{'='*60}")
        
        try:
            # Load damaged signals
            damaged_signals_np = load_sensor_data(damage_path, sensor_names)
            damaged_signals = prepare_signals_for_pso(damaged_signals_np).to(device)
            
            # Initialize PSO
            pso = BridgePSO(
                n_particles=n_particles,
                n_elements=len(sensor_names),
                bounds=(0.1, 1.0),
                c1=0.5,
                c2=0.3,
                w=0.9,
                device=device
            )
            
            # Run optimization
            best_damage = pso.optimize(
                healthy_signals,
                damaged_signals,
                n_iterations=n_iterations,
                verbose=True
            )
            
            # Convert to CPU for analysis
            predicted_damage = best_damage.cpu().numpy()
            
            # Store results
            result = {
                'damage_case': damage_path.split('/')[-2],
                'predicted_stiffness': predicted_damage.tolist(),
                'damage_severity': (1.0 - predicted_damage).tolist(),
                'final_fitness': pso.global_best_score,
                'sensor_names': sensor_names
            }
            results.append(result)
            
            # Plot results
            plot_pso_results(
                pso.history,
                predicted_damage,
                sensor_names,
                save_path=f"pso_result_{idx+1}.png"
            )
            
            print(f"\nResults for {result['damage_case']}:")
            for sensor, stiffness, damage in zip(sensor_names, predicted_damage, 1-predicted_damage):
                status = "DAMAGED" if damage > 0.1 else "HEALTHY"
                print(f"  {sensor}: Stiffness={stiffness:.3f}, Damage={damage:.3f} [{status}]")
            
        except Exception as e:
            print(f"Error processing {damage_path}: {e}")
            continue
    
    # Save all results
    results_df = pd.DataFrame([
        {
            'damage_case': r['damage_case'],
            **{f'{sensor}_stiffness': s for sensor, s in zip(sensor_names, r['predicted_stiffness'])},
            **{f'{sensor}_damage': d for sensor, d in zip(sensor_names, r['damage_severity'])},
            'final_fitness': r['final_fitness']
        }
        for r in results
    ])
    
    results_df.to_csv('pso_damage_localization_results.csv', index=False)
    print("\n" + "="*60)
    print("Results saved to: pso_damage_localization_results.csv")
    print("="*60)
    
    return results

# Main Execution

In [ ]:
def main():
    # Paths (adjust for Kaggle environment)
    healthy_path = '/kaggle/input/dojo-dataset/complete_simulation_dataset/healthy/2017-11-03.csv'
    
    # Collect damage case paths
    damage_paths = []
    for i in range(1, 9):
        pattern = f'/kaggle/input/dojo-dataset/simulation_results (1)/damage_case_{i}_*/2018-07-17_1.csv'
        damage_paths.extend(glob(pattern))
    
    print(f"Found {len(damage_paths)} damage case files")
    
    # Run PSO
    results = run_pso_on_dataset(
        healthy_path=healthy_path,
        damage_paths=damage_paths,
        n_particles=50,
        n_iterations=50,
        max_samples=10,  # Process first 10 damage cases
        device='cuda'
    )
    
    print("\n✅ PSO optimization complete!")
    print(f"Processed {len(results)} damage cases")

if __name__ == "__main__":
    main()

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from glob import glob
import warnings
from scipy.fft import fft, fftfreq
import matplotlib.pyplot as plt
from tqdm import tqdm
import json
import os
from pathlib import Path

warnings.filterwarnings('ignore')

# ============================================================================
# GPU-ACCELERATED PSO IMPLEMENTATION
# ============================================================================

class BridgePSO:
    """GPU-accelerated Particle Swarm Optimization for damage localization"""
    
    def __init__(self, n_particles=50, n_elements=3, bounds=(0.1, 1.0), 
                 c1=0.5, c2=0.3, w=0.9, device='cuda'):
        self.n_particles = n_particles
        self.n_elements = n_elements
        self.device = torch.device(device if torch.cuda.is_available() else 'cpu')
        
        self.c1 = c1
        self.c2 = c2
        self.w = w
        self.bounds = bounds
        
        # Initialize particles and velocities
        self.positions = torch.rand(n_particles, n_elements, device=self.device)
        self.positions = self.positions * (bounds[1] - bounds[0]) + bounds[0]
        self.velocities = torch.randn(n_particles, n_elements, device=self.device) * 0.1
        
        # Best positions
        self.personal_best_positions = self.positions.clone()
        self.personal_best_scores = torch.full((n_particles,), float('inf'), device=self.device)
        self.global_best_position = None
        self.global_best_score = float('inf')
        
        # History tracking
        self.history = {
            'global_best_scores': [],
            'mean_scores': [],
            'std_scores': []
        }
    
    def apply_damage_to_signals(self, healthy_signals, damage_factors):
        """Apply damage factors to healthy signals"""
        damaged = healthy_signals.clone()
        for i, factor in enumerate(damage_factors):
            damaged[i] *= factor
            noise_level = (1 - factor) * 0.1
            damaged[i] += torch.randn_like(damaged[i]) * noise_level * damaged[i].std()
        return damaged
    
    def fitness_function(self, particles, healthy_signals, observed_signals):
        """Evaluate fitness for all particles (GPU-accelerated)"""
        batch_size = particles.shape[0]
        fitness_scores = torch.zeros(batch_size, device=self.device)
        
        for i in range(batch_size):
            simulated = self.apply_damage_to_signals(healthy_signals, particles[i])
            error = torch.sqrt(torch.mean((simulated - observed_signals) ** 2))
            sparsity_penalty = torch.sum(1.0 - particles[i]) * 0.01
            fitness_scores[i] = error + sparsity_penalty
        
        return fitness_scores
    
    def update(self, fitness_scores):
        """Update particle positions and velocities"""
        # Update personal bests
        improved = fitness_scores < self.personal_best_scores
        self.personal_best_scores[improved] = fitness_scores[improved]
        self.personal_best_positions[improved] = self.positions[improved].clone()
        
        # Update global best
        min_idx = torch.argmin(fitness_scores)
        if fitness_scores[min_idx] < self.global_best_score:
            self.global_best_score = fitness_scores[min_idx].item()
            self.global_best_position = self.positions[min_idx].clone()
        
        # Update velocities and positions
        r1 = torch.rand_like(self.positions)
        r2 = torch.rand_like(self.positions)
        
        cognitive = self.c1 * r1 * (self.personal_best_positions - self.positions)
        social = self.c2 * r2 * (self.global_best_position - self.positions)
        
        self.velocities = self.w * self.velocities + cognitive + social
        self.positions = self.positions + self.velocities
        self.positions = torch.clamp(self.positions, self.bounds[0], self.bounds[1])
    
    def optimize(self, healthy_signals, observed_signals, n_iterations=50, verbose=True):
        """Run PSO optimization"""
        iterator = tqdm(range(n_iterations), disable=not verbose)
        
        for iteration in iterator:
            fitness_scores = self.fitness_function(self.positions, healthy_signals, observed_signals)
            self.update(fitness_scores)
            
            self.history['global_best_scores'].append(self.global_best_score)
            self.history['mean_scores'].append(fitness_scores.mean().item())
            self.history['std_scores'].append(fitness_scores.std().item())
            
            if verbose and iteration % 10 == 0:
                iterator.set_description(f"Best Fitness: {self.global_best_score:.6f}")
        
        return self.global_best_position

# ============================================================================
# DATA LOADING
# ============================================================================

def load_sensor_data(csv_path, sensor_names=['Sensor_24', 'Sensor_31', 'Sensor_32']):
    """Load sensor data from CSV file"""
    try:
        df = pd.read_csv(csv_path)
        signals = []
        for sensor in sensor_names:
            sensor_cols = [col for col in df.columns if sensor in col]
            if len(sensor_cols) > 0:
                signals.append(df[sensor_cols[0]].values)
        
        return np.array(signals) if len(signals) == len(sensor_names) else None
    except Exception as e:
        return None

def create_synthetic_damage(healthy_signals, damage_pattern):
    """Create synthetic damage scenarios"""
    device = healthy_signals.device
    n_sensors = healthy_signals.shape[0]
    
    true_damage_factors = torch.ones(n_sensors, device=device)
    for sensor_idx in damage_pattern['sensors']:
        if sensor_idx < n_sensors:
            true_damage_factors[sensor_idx] = damage_pattern['severity']
    
    damaged_signals = healthy_signals.clone()
    for i, factor in enumerate(true_damage_factors):
        damaged_signals[i] *= factor
        noise_level = (1 - factor) * 0.08
        damaged_signals[i] += torch.randn_like(damaged_signals[i]) * noise_level * damaged_signals[i].std()
    
    return damaged_signals, true_damage_factors

# ============================================================================
# EVALUATION
# ============================================================================

def evaluate_pso_results(predicted_damage, true_damage, threshold=0.9):
    """Evaluate PSO damage localization results"""
    pred_damaged = predicted_damage < threshold
    true_damaged = true_damage < threshold
    
    tp = np.sum(pred_damaged & true_damaged)
    fp = np.sum(pred_damaged & ~true_damaged)
    tn = np.sum(~pred_damaged & ~true_damaged)
    fn = np.sum(~pred_damaged & true_damaged)
    
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    
    return {
        'accuracy': accuracy, 'precision': precision, 'recall': recall, 'f1_score': f1,
        'true_positives': tp, 'false_positives': fp, 'true_negatives': tn, 'false_negatives': fn
    }

# ============================================================================
# VISUALIZATION
# ============================================================================

def plot_pso_results(history, predicted_damage, true_damage, sensor_names, metrics, case_name, save_path):
    """Plot comprehensive PSO optimization results"""
    fig = plt.figure(figsize=(18, 10))
    gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)
    
    # Plot 1: Fitness convergence
    ax1 = fig.add_subplot(gs[0, :2])
    ax1.plot(history['global_best_scores'], label='Best Fitness', linewidth=2.5, color='#2E86AB')
    ax1.plot(history['mean_scores'], label='Mean Fitness', alpha=0.7, linewidth=2, color='#A23B72')
    ax1.fill_between(range(len(history['mean_scores'])),
                     np.array(history['mean_scores']) - np.array(history['std_scores']),
                     np.array(history['mean_scores']) + np.array(history['std_scores']),
                     alpha=0.2, color='#A23B72')
    ax1.set_xlabel('Iteration', fontsize=12, fontweight='bold')
    ax1.set_ylabel('Fitness (RMS Error)', fontsize=12, fontweight='bold')
    ax1.set_title(f'PSO Convergence - {case_name}', fontsize=14, fontweight='bold')
    ax1.legend(fontsize=10)
    ax1.grid(True, alpha=0.3)
    
    # Plot 2: Performance Metrics
    ax2 = fig.add_subplot(gs[0, 2])
    metric_names = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
    metric_values = [metrics['accuracy'], metrics['precision'], metrics['recall'], metrics['f1_score']]
    colors_metrics = ['#06A77D', '#F18F01', '#C73E1D', '#6A4C93']
    bars = ax2.barh(metric_names, metric_values, color=colors_metrics, alpha=0.8, edgecolor='black')
    ax2.set_xlim(0, 1)
    ax2.set_xlabel('Score', fontsize=11, fontweight='bold')
    ax2.set_title('Performance Metrics', fontsize=12, fontweight='bold')
    ax2.grid(True, alpha=0.3, axis='x')
    for bar, val in zip(bars, metric_values):
        ax2.text(bar.get_width() + 0.02, bar.get_y() + bar.get_height()/2.,
                f'{val:.3f}', ha='left', va='center', fontweight='bold')
    
    # Plot 3: Stiffness Comparison
    ax3 = fig.add_subplot(gs[1, :2])
    x = np.arange(len(sensor_names))
    width = 0.35
    pred_stiffness = predicted_damage
    true_stiffness = true_damage.cpu().numpy() if isinstance(true_damage, torch.Tensor) else true_damage
    
    bars1 = ax3.bar(x - width/2, true_stiffness, width, label='True Stiffness', 
                    color='#06A77D', alpha=0.8, edgecolor='black')
    bars2 = ax3.bar(x + width/2, pred_stiffness, width, label='Predicted Stiffness',
                    color='#F18F01', alpha=0.8, edgecolor='black')
    ax3.set_ylabel('Stiffness Factor', fontsize=12, fontweight='bold')
    ax3.set_title('Stiffness Comparison: True vs Predicted', fontsize=13, fontweight='bold')
    ax3.set_xticks(x)
    ax3.set_xticklabels(sensor_names, rotation=45, ha='right')
    ax3.legend(fontsize=10)
    ax3.grid(True, alpha=0.3, axis='y')
    ax3.set_ylim(0, 1.1)
    
    for bars in [bars1, bars2]:
        for bar in bars:
            height = bar.get_height()
            ax3.text(bar.get_x() + bar.get_width()/2., height + 0.02,
                    f'{height:.2f}', ha='center', va='bottom', fontsize=9)
    
    # Plot 4: Damage Severity
    ax4 = fig.add_subplot(gs[1, 2])
    true_damage_sev = 1.0 - true_stiffness
    pred_damage_sev = 1.0 - pred_stiffness
    
    bars1 = ax4.bar(x - width/2, true_damage_sev, width, label='True Damage',
                    color='#C73E1D', alpha=0.8, edgecolor='black')
    bars2 = ax4.bar(x + width/2, pred_damage_sev, width, label='Predicted Damage',
                    color='#6A4C93', alpha=0.8, edgecolor='black')
    ax4.axhline(y=0.1, color='orange', linestyle='--', linewidth=2, label='Damage Threshold')
    ax4.set_ylabel('Damage Severity', fontsize=11, fontweight='bold')
    ax4.set_title('Damage Severity', fontsize=12, fontweight='bold')
    ax4.set_xticks(x)
    ax4.set_xticklabels(sensor_names, rotation=45, ha='right')
    ax4.legend(fontsize=9)
    ax4.grid(True, alpha=0.3, axis='y')
    
    # Plot 5: Error Analysis
    ax5 = fig.add_subplot(gs[2, :])
    stiffness_error = np.abs(pred_stiffness - true_stiffness)
    damage_error = np.abs(pred_damage_sev - true_damage_sev)
    
    bars1 = ax5.bar(x - width/2, stiffness_error, width, label='Stiffness Error',
                    color='#2E86AB', alpha=0.8, edgecolor='black')
    bars2 = ax5.bar(x + width/2, damage_error, width, label='Damage Severity Error',
                    color='#A23B72', alpha=0.8, edgecolor='black')
    ax5.set_ylabel('Absolute Error', fontsize=12, fontweight='bold')
    ax5.set_xlabel('Sensors', fontsize=12, fontweight='bold')
    ax5.set_title('Prediction Error Analysis', fontsize=13, fontweight='bold')
    ax5.set_xticks(x)
    ax5.set_xticklabels(sensor_names)
    ax5.legend(fontsize=10)
    ax5.grid(True, alpha=0.3, axis='y')
    
    for bars in [bars1, bars2]:
        for bar in bars:
            height = bar.get_height()
            if height > 0.01:
                ax5.text(bar.get_x() + bar.get_width()/2., height + 0.005,
                        f'{height:.3f}', ha='center', va='bottom', fontsize=8)
    
    plt.suptitle(f'PSO Damage Localization Results - {case_name}', 
                fontsize=16, fontweight='bold', y=0.995)
    plt.savefig(save_path, dpi=300, bbox_inches='tight', facecolor='white')
    plt.close()
    print(f"   ✅ Graph saved: {save_path}")

def plot_overall_performance(all_metrics, scenarios, save_path):
    """Plot overall performance across all scenarios"""
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    scenario_names = [s['name'] for s in scenarios]
    
    # Plot 1: Accuracy across scenarios
    ax = axes[0, 0]
    accuracies = [m['accuracy'] for m in all_metrics]
    colors = ['#06A77D' if acc > 0.8 else '#F18F01' if acc > 0.6 else '#C73E1D' for acc in accuracies]
    ax.bar(range(len(accuracies)), accuracies, color=colors, alpha=0.8, edgecolor='black')
    ax.set_ylabel('Accuracy', fontsize=12, fontweight='bold')
    ax.set_title('Accuracy Across Scenarios', fontsize=13, fontweight='bold')
    ax.set_xticks(range(len(scenario_names)))
    ax.set_xticklabels(scenario_names, rotation=45, ha='right', fontsize=8)
    ax.axhline(y=0.8, color='green', linestyle='--', alpha=0.5, label='Good (0.8)')
    ax.axhline(y=0.6, color='orange', linestyle='--', alpha=0.5, label='Fair (0.6)')
    ax.set_ylim(0, 1.1)
    ax.legend()
    ax.grid(True, alpha=0.3, axis='y')
    
    # Plot 2: All metrics comparison
    ax = axes[0, 1]
    x = np.arange(len(all_metrics))
    width = 0.2
    precisions = [m['precision'] for m in all_metrics]
    recalls = [m['recall'] for m in all_metrics]
    f1_scores = [m['f1_score'] for m in all_metrics]
    
    ax.bar(x - width, accuracies, width, label='Accuracy', color='#06A77D', alpha=0.8)
    ax.bar(x, precisions, width, label='Precision', color='#F18F01', alpha=0.8)
    ax.bar(x + width, recalls, width, label='Recall', color='#C73E1D', alpha=0.8)
    ax.bar(x + 2*width, f1_scores, width, label='F1-Score', color='#6A4C93', alpha=0.8)
    ax.set_ylabel('Score', fontsize=12, fontweight='bold')
    ax.set_title('All Metrics Comparison', fontsize=13, fontweight='bold')
    ax.set_xticks(x + width/2)
    ax.set_xticklabels(range(1, len(all_metrics)+1))
    ax.set_xlabel('Scenario Number', fontsize=11, fontweight='bold')
    ax.legend()
    ax.grid(True, alpha=0.3, axis='y')
    ax.set_ylim(0, 1.1)
    
    # Plot 3: Confusion Matrix
    ax = axes[1, 0]
    total_tp = sum(m['true_positives'] for m in all_metrics)
    total_fp = sum(m['false_positives'] for m in all_metrics)
    total_tn = sum(m['true_negatives'] for m in all_metrics)
    total_fn = sum(m['false_negatives'] for m in all_metrics)
    
    confusion = np.array([[total_tn, total_fp], [total_fn, total_tp]])
    im = ax.imshow(confusion, cmap='Blues', alpha=0.8)
    ax.set_xticks([0, 1])
    ax.set_yticks([0, 1])
    ax.set_xticklabels(['Predicted Healthy', 'Predicted Damaged'], fontsize=10)
    ax.set_yticklabels(['True Healthy', 'True Damaged'], fontsize=10)
    ax.set_title('Cumulative Confusion Matrix', fontsize=13, fontweight='bold')
    
    for i in range(2):
        for j in range(2):
            ax.text(j, i, confusion[i, j], ha="center", va="center", 
                   color="black", fontsize=20, fontweight='bold')
    plt.colorbar(im, ax=ax)
    
    # Plot 4: Distribution
    ax = axes[1, 1]
    metrics_data = [accuracies, precisions, recalls, f1_scores]
    positions = [1, 2, 3, 4]
    ax.boxplot(metrics_data, positions=positions, widths=0.6, patch_artist=True,
               boxprops=dict(facecolor='lightblue', alpha=0.7),
               medianprops=dict(color='red', linewidth=2))
    ax.set_xticklabels(['Accuracy', 'Precision', 'Recall', 'F1-Score'], fontsize=11)
    ax.set_ylabel('Score', fontsize=12, fontweight='bold')
    ax.set_title('Performance Metrics Distribution', fontsize=13, fontweight='bold')
    ax.grid(True, alpha=0.3, axis='y')
    ax.set_ylim(0, 1.1)
    
    means = [np.mean(data) for data in metrics_data]
    ax.plot(positions, means, 'D', color='green', markersize=10, label='Mean', zorder=3)
    ax.legend()
    
    plt.suptitle('PSO Damage Localization - Overall Performance Analysis', 
                fontsize=16, fontweight='bold', y=0.995)
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight', facecolor='white')
    plt.close()
    print(f"   ✅ Overall performance graph: {save_path}")

# ============================================================================
# MAIN PIPELINE
# ============================================================================

def run_pso_simulation(healthy_paths=None, healthy_signals_tensor=None, n_particles=50, 
                      n_iterations=50, n_damage_scenarios=15, device='cuda', output_dir='pso_results'):
    """Run PSO simulation on multiple healthy datasets"""
    os.makedirs(output_dir, exist_ok=True)
    device = torch.device(device if torch.cuda.is_available() else 'cpu')
    print(f"🚀 Using device: {device}")
    
    sensor_names = ['Sensor_24', 'Sensor_31', 'Sensor_32']
    max_length = 1000
    
    # Check if pre-computed tensor is provided
    if healthy_signals_tensor is not None:
        healthy_signals = healthy_signals_tensor.to(device)
        print(f"\n📊 Using pre-computed healthy signals tensor")
        print(f"   📏 Healthy signals shape: {healthy_signals.shape}")
    else:
        # Load and normalize all healthy signals from files
        print(f"\n📊 Loading {len(healthy_paths)} healthy datasets...")
        healthy_signals_list = []
        
        for path in tqdm(healthy_paths):
            signals = load_sensor_data(path, sensor_names)
            if signals is not None:
                normalized_signals = []
                for signal in signals:
                    if len(signal) > max_length:
                        signal = signal[:max_length]
                    elif len(signal) < max_length:
                        signal = np.pad(signal, (0, max_length - len(signal)), mode='constant')
                    normalized_signals.append(signal)
                healthy_signals_list.append(np.array(normalized_signals))
        
        print(f"   ✅ Successfully loaded {len(healthy_signals_list)} healthy datasets")
        
        if len(healthy_signals_list) == 0:
            raise ValueError("No healthy datasets loaded! Please provide valid CSV files or a healthy_signals_tensor.")
        
        # Calculate ensemble baseline
        healthy_signals_array = np.array(healthy_signals_list)
        healthy_baseline = np.mean(healthy_signals_array, axis=0)
        healthy_signals = torch.tensor(healthy_baseline, dtype=torch.float32).to(device)
        print(f"   📏 Healthy signals shape: {healthy_signals.shape}")
    
    # Define damage scenarios
    damage_scenarios = [
        {'name': 'Single_Sensor24_Severe', 'sensors': [0], 'severity': 0.3},
        {'name': 'Single_Sensor24_Moderate', 'sensors': [0], 'severity': 0.6},
        {'name': 'Single_Sensor31_Severe', 'sensors': [1], 'severity': 0.3},
        {'name': 'Single_Sensor31_Moderate', 'sensors': [1], 'severity': 0.6},
        {'name': 'Single_Sensor32_Severe', 'sensors': [2], 'severity': 0.3},
        {'name': 'Single_Sensor32_Moderate', 'sensors': [2], 'severity': 0.6},
        {'name': 'Double_S24_S31_Severe', 'sensors': [0, 1], 'severity': 0.4},
        {'name': 'Double_S24_S31_Moderate', 'sensors': [0, 1], 'severity': 0.7},
        {'name': 'Double_S24_S32_Severe', 'sensors': [0, 2], 'severity': 0.4},
        {'name': 'Double_S31_S32_Moderate', 'sensors': [1, 2], 'severity': 0.7},
        {'name': 'Triple_All_Mild', 'sensors': [0, 1, 2], 'severity': 0.8},
        {'name': 'Triple_All_Moderate', 'sensors': [0, 1, 2], 'severity': 0.6},
        {'name': 'Triple_All_Severe', 'sensors': [0, 1, 2], 'severity': 0.4},
        {'name': 'Single_Sensor24_Mild', 'sensors': [0], 'severity': 0.85},
        {'name': 'Single_Sensor32_Mild', 'sensors': [2], 'severity': 0.85},
    ]
    
    results = []
    all_metrics = []
    
    print(f"\n🔬 Running PSO on {len(damage_scenarios[:n_damage_scenarios])} damage scenarios...\n")
    
    for idx, scenario in enumerate(damage_scenarios[:n_damage_scenarios]):
        print(f"{'='*70}")
        print(f"🔧 Scenario {idx+1}/{min(n_damage_scenarios, len(damage_scenarios))}: {scenario['name']}")
        print(f"{'='*70}")
        
        try:
            damaged_signals, true_damage = create_synthetic_damage(healthy_signals, scenario)
            print(f"   🎯 True damage: {(1 - true_damage.cpu().numpy()).round(3).tolist()}")
            
            pso = BridgePSO(n_particles=n_particles, n_elements=len(sensor_names),
                          bounds=(0.1, 1.0), c1=0.5, c2=0.3, w=0.9, device=device)
            
            best_damage = pso.optimize(healthy_signals, damaged_signals, n_iterations, verbose=True)
            predicted_damage = best_damage.cpu().numpy()
            metrics = evaluate_pso_results(predicted_damage, true_damage.cpu().numpy())
            
            print(f"\n   📊 Metrics: Acc={metrics['accuracy']:.3f}, Prec={metrics['precision']:.3f}, "
                  f"Recall={metrics['recall']:.3f}, F1={metrics['f1_score']:.3f}")
            
            result = {
                'scenario': scenario['name'],
                'true_damage_sensors': scenario['sensors'],
                'true_severity': scenario['severity'],
                'predicted_stiffness': predicted_damage.tolist(),
                'predicted_damage_severity': (1.0 - predicted_damage).tolist(),
                'final_fitness': pso.global_best_score,
                'sensor_names': sensor_names,
                **metrics
            }
            results.append(result)
            all_metrics.append(metrics)
            
            save_path = os.path.join(output_dir, f"pso_scenario_{idx+1:02d}_{scenario['name']}.png")
            plot_pso_results(pso.history, predicted_damage, true_damage, sensor_names, 
                           metrics, scenario['name'], save_path)
            
            print(f"\n   ✨ Predictions:")
            for sensor, pred_stiff, pred_damage in zip(sensor_names, predicted_damage, 1-predicted_damage):
                status = "🔴 DAMAGED" if pred_damage > 0.1 else "🟢 HEALTHY"
                print(f"      {sensor}: Stiffness={pred_stiff:.3f}, Damage={pred_damage:.3f} [{status}]")
            
        except Exception as e:
            print(f"   ❌ Error: {e}")
            continue
    
    # Save results
    print(f"\n{'='*70}")
    print("💾 Saving results...")
    print(f"{'='*70}")
    
    results_df = pd.DataFrame([{
        'scenario': r['scenario'],
        'true_damage_sensors': str(r['true_damage_sensors']),
        'true_severity': r['true_severity'],
        **{f'{sensor}_pred_stiffness': s for sensor, s in zip(sensor_names, r['predicted_stiffness'])},
        **{f'{sensor}_pred_damage': d for sensor, d in zip(sensor_names, r['predicted_damage_severity'])},
        'final_fitness': r['final_fitness'],
        'accuracy': r['accuracy'],
        'precision': r['precision'],
        'recall': r['recall'],
        'f1_score': r['f1_score']
    } for r in results])
    
    csv_path = os.path.join(output_dir, 'pso_damage_localization_results.csv')
    results_df.to_csv(csv_path, index=False)
    print(f"   ✅ Results CSV: {csv_path}")
    
    summary_stats = {
        'total_scenarios': len(results),
        'mean_accuracy': np.mean([m['accuracy'] for m in all_metrics]),
        'mean_precision': np.mean([m['precision'] for m in all_metrics]),
        'mean_recall': np.mean([m['recall'] for m in all_metrics]),
        'mean_f1_score': np.mean([m['f1_score'] for m in all_metrics]),
        'std_accuracy': np.std([m['accuracy'] for m in all_metrics]),
        'std_precision': np.std([m['precision'] for m in all_metrics]),
        'std_recall': np.std([m['recall'] for m in all_metrics]),
        'std_f1_score': np.std([m['f1_score'] for m in all_metrics])
    }
    
    json_path = os.path.join(output_dir, 'pso_summary_statistics.json')
    with open(json_path, 'w') as f:
        json.dump(summary_stats, f, indent=4)
    print(f"   ✅ Summary JSON: {json_path}")
    
    plot_overall_performance(all_metrics, damage_scenarios[:n_damage_scenarios], 
                           os.path.join(output_dir, 'pso_overall_performance.png'))
    
    print(f"\n{'='*70}")
    print("✅ PSO SIMULATION COMPLETE!")
    print(f"{'='*70}")
    print(f"\n📈 Overall Performance:")
    print(f"   Mean Accuracy:  {summary_stats['mean_accuracy']:.4f} ± {summary_stats['std_accuracy']:.4f}")
    print(f"   Mean Precision: {summary_stats['mean_precision']:.4f} ± {summary_stats['std_precision']:.4f}")
    print(f"   Mean Recall:    {summary_stats['mean_recall']:.4f} ± {summary_stats['std_recall']:.4f}")
    print(f"   Mean F1-Score:  {summary_stats['mean_f1_score']:.4f} ± {summary_stats['std_f1_score']:.4f}")
    
    return results, summary_stats

# ============================================================================
# EXECUTION
# ============================================================================

if __name__ == "__main__":
    # Example usage
    print("="*70)
    print("  PSO-BASED BRIDGE DAMAGE LOCALIZATION SYSTEM")
    print("="*70)
    
    # Find all healthy CSV files
    healthy_csv_pattern = "healthy_*.csv"  # Modify this pattern as needed
    healthy_paths = glob(healthy_csv_pattern)
    
    if len(healthy_paths) == 0:
        print("\n⚠️  No healthy CSV files found!")
        print("   Please place healthy sensor data CSV files in the current directory")
        print("   Expected pattern: healthy_*.csv")
        print("\n   Creating synthetic demo instead...")
        
        # Create synthetic demo data
        n_sensors = 3
        n_samples = 1000
        synthetic_signals = []
        
        for i in range(5):  # 5 synthetic healthy datasets
            sensor_data = []
            for s in range(n_sensors):
                # Generate synthetic vibration signals
                t = np.linspace(0, 10, n_samples)
                freq = 2 + s * 0.5
                signal = np.sin(2 * np.pi * freq * t) + np.random.normal(0, 0.1, n_samples)
                sensor_data.append(signal)
            synthetic_signals.append(np.array(sensor_data))
        
        # Convert to tensor
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        healthy_baseline = np.mean(synthetic_signals, axis=0)
        healthy_signals = torch.tensor(healthy_baseline, dtype=torch.float32).to(device)
        
        print(f"\n✅ Created synthetic healthy signals: {healthy_signals.shape}")
        print(f"🚀 Running PSO with synthetic data...\n")
        
        # Run simulation with synthetic data
        results, summary = run_pso_simulation(
            healthy_signals_tensor=healthy_signals,  # Pass the pre-computed tensor
            n_particles=30,
            n_iterations=40,
            n_damage_scenarios=10,
            device='cuda',
            output_dir='pso_results_synthetic'
        )
        
    else:
        print(f"\n✅ Found {len(healthy_paths)} healthy CSV files")
        print("   Files:", [Path(p).name for p in healthy_paths[:5]], "..." if len(healthy_paths) > 5 else "")
        
        # Run simulation with real data
        results, summary = run_pso_simulation(
            healthy_paths=healthy_paths,
            n_particles=50,
            n_iterations=50,
            n_damage_scenarios=15,
            device='cuda',
            output_dir='pso_results'
        )
    
    print("\n" + "="*70)
    print("  SIMULATION COMPLETED SUCCESSFULLY!")
    print("="*70)
    print("\n📁 Check the output directory for:")
    print("   • Individual scenario plots")
    print("   • Overall performance analysis")
    print("   • Results CSV file")
    print("   • Summary statistics JSON")
    print("\n🎉 All done!")

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from glob import glob
import warnings
from scipy.fft import fft, fftfreq
import matplotlib.pyplot as plt
from tqdm import tqdm
import json
import os
from pathlib import Path

warnings.filterwarnings('ignore')

# ============================================================================
# GPU-ACCELERATED PSO IMPLEMENTATION
# ============================================================================

class BridgePSO:
    """GPU-accelerated Particle Swarm Optimization for damage localization"""
    
    def __init__(self, n_particles=50, n_elements=3, bounds=(0.1, 1.0), 
                 c1=0.5, c2=0.3, w=0.9, device='cuda'):
        self.n_particles = n_particles
        self.n_elements = n_elements
        self.device = torch.device(device if torch.cuda.is_available() else 'cpu')
        
        self.c1 = c1
        self.c2 = c2
        self.w = w
        self.bounds = bounds
        
        # Initialize particles and velocities
        self.positions = torch.rand(n_particles, n_elements, device=self.device)
        self.positions = self.positions * (bounds[1] - bounds[0]) + bounds[0]
        self.velocities = torch.randn(n_particles, n_elements, device=self.device) * 0.1
        
        # Best positions
        self.personal_best_positions = self.positions.clone()
        self.personal_best_scores = torch.full((n_particles,), float('inf'), device=self.device)
        self.global_best_position = None
        self.global_best_score = float('inf')
        
        # History tracking
        self.history = {
            'global_best_scores': [],
            'mean_scores': [],
            'std_scores': []
        }
    
    def apply_damage_to_signals(self, healthy_signals, damage_factors):
        """Apply damage factors to healthy signals"""
        damaged = healthy_signals.clone()
        for i, factor in enumerate(damage_factors):
            damaged[i] *= factor
            noise_level = (1 - factor) * 0.1
            damaged[i] += torch.randn_like(damaged[i]) * noise_level * damaged[i].std()
        return damaged
    
    def fitness_function(self, particles, healthy_signals, observed_signals):
        """Evaluate fitness for all particles (GPU-accelerated)"""
        batch_size = particles.shape[0]
        fitness_scores = torch.zeros(batch_size, device=self.device)
        
        for i in range(batch_size):
            simulated = self.apply_damage_to_signals(healthy_signals, particles[i])
            error = torch.sqrt(torch.mean((simulated - observed_signals) ** 2))
            sparsity_penalty = torch.sum(1.0 - particles[i]) * 0.01
            fitness_scores[i] = error + sparsity_penalty
        
        return fitness_scores
    
    def update(self, fitness_scores):
        """Update particle positions and velocities"""
        # Update personal bests
        improved = fitness_scores < self.personal_best_scores
        self.personal_best_scores[improved] = fitness_scores[improved]
        self.personal_best_positions[improved] = self.positions[improved].clone()
        
        # Update global best
        min_idx = torch.argmin(fitness_scores)
        if fitness_scores[min_idx] < self.global_best_score:
            self.global_best_score = fitness_scores[min_idx].item()
            self.global_best_position = self.positions[min_idx].clone()
        
        # Update velocities and positions
        r1 = torch.rand_like(self.positions)
        r2 = torch.rand_like(self.positions)
        
        cognitive = self.c1 * r1 * (self.personal_best_positions - self.positions)
        social = self.c2 * r2 * (self.global_best_position - self.positions)
        
        self.velocities = self.w * self.velocities + cognitive + social
        self.positions = self.positions + self.velocities
        self.positions = torch.clamp(self.positions, self.bounds[0], self.bounds[1])
    
    def optimize(self, healthy_signals, observed_signals, n_iterations=50, verbose=True):
        """Run PSO optimization"""
        iterator = tqdm(range(n_iterations), disable=not verbose)
        
        for iteration in iterator:
            fitness_scores = self.fitness_function(self.positions, healthy_signals, observed_signals)
            self.update(fitness_scores)
            
            self.history['global_best_scores'].append(self.global_best_score)
            self.history['mean_scores'].append(fitness_scores.mean().item())
            self.history['std_scores'].append(fitness_scores.std().item())
            
            if verbose and iteration % 10 == 0:
                iterator.set_description(f"Best Fitness: {self.global_best_score:.6f}")
        
        return self.global_best_position

# ============================================================================
# DATA LOADING
# ============================================================================

def load_sensor_data(csv_path, sensor_names=['Sensor_24', 'Sensor_31', 'Sensor_32']):
    """Load sensor data from CSV file"""
    try:
        df = pd.read_csv(csv_path)
        signals = []
        for sensor in sensor_names:
            sensor_cols = [col for col in df.columns if sensor in col]
            if len(sensor_cols) > 0:
                signals.append(df[sensor_cols[0]].values)
        
        return np.array(signals) if len(signals) == len(sensor_names) else None
    except Exception as e:
        return None

def create_synthetic_damage(healthy_signals, damage_pattern):
    """Create synthetic damage scenarios"""
    device = healthy_signals.device
    n_sensors = healthy_signals.shape[0]
    
    true_damage_factors = torch.ones(n_sensors, device=device)
    for sensor_idx in damage_pattern['sensors']:
        if sensor_idx < n_sensors:
            true_damage_factors[sensor_idx] = damage_pattern['severity']
    
    damaged_signals = healthy_signals.clone()
    for i, factor in enumerate(true_damage_factors):
        damaged_signals[i] *= factor
        noise_level = (1 - factor) * 0.08
        damaged_signals[i] += torch.randn_like(damaged_signals[i]) * noise_level * damaged_signals[i].std()
    
    return damaged_signals, true_damage_factors

# ============================================================================
# EVALUATION
# ============================================================================

def evaluate_pso_results(predicted_damage, true_damage, threshold=0.9):
    """Evaluate PSO damage localization results"""
    pred_damaged = predicted_damage < threshold
    true_damaged = true_damage < threshold
    
    tp = np.sum(pred_damaged & true_damaged)
    fp = np.sum(pred_damaged & ~true_damaged)
    tn = np.sum(~pred_damaged & ~true_damaged)
    fn = np.sum(~pred_damaged & true_damaged)
    
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    
    return {
        'accuracy': accuracy, 'precision': precision, 'recall': recall, 'f1_score': f1,
        'true_positives': tp, 'false_positives': fp, 'true_negatives': tn, 'false_negatives': fn
    }

# ============================================================================
# VISUALIZATION
# ============================================================================

def plot_pso_results(history, predicted_damage, true_damage, sensor_names, metrics, case_name, save_path):
    """Plot comprehensive PSO optimization results"""
    fig = plt.figure(figsize=(18, 10))
    gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)
    
    # Plot 1: Fitness convergence
    ax1 = fig.add_subplot(gs[0, :2])
    ax1.plot(history['global_best_scores'], label='Best Fitness', linewidth=2.5, color='#2E86AB')
    ax1.plot(history['mean_scores'], label='Mean Fitness', alpha=0.7, linewidth=2, color='#A23B72')
    ax1.fill_between(range(len(history['mean_scores'])),
                     np.array(history['mean_scores']) - np.array(history['std_scores']),
                     np.array(history['mean_scores']) + np.array(history['std_scores']),
                     alpha=0.2, color='#A23B72')
    ax1.set_xlabel('Iteration', fontsize=12, fontweight='bold')
    ax1.set_ylabel('Fitness (RMS Error)', fontsize=12, fontweight='bold')
    ax1.set_title(f'PSO Convergence - {case_name}', fontsize=14, fontweight='bold')
    ax1.legend(fontsize=10)
    ax1.grid(True, alpha=0.3)
    
    # Plot 2: Performance Metrics
    ax2 = fig.add_subplot(gs[0, 2])
    metric_names = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
    metric_values = [metrics['accuracy'], metrics['precision'], metrics['recall'], metrics['f1_score']]
    colors_metrics = ['#06A77D', '#F18F01', '#C73E1D', '#6A4C93']
    bars = ax2.barh(metric_names, metric_values, color=colors_metrics, alpha=0.8, edgecolor='black')
    ax2.set_xlim(0, 1)
    ax2.set_xlabel('Score', fontsize=11, fontweight='bold')
    ax2.set_title('Performance Metrics', fontsize=12, fontweight='bold')
    ax2.grid(True, alpha=0.3, axis='x')
    for bar, val in zip(bars, metric_values):
        ax2.text(bar.get_width() + 0.02, bar.get_y() + bar.get_height()/2.,
                f'{val:.3f}', ha='left', va='center', fontweight='bold')
    
    # Plot 3: Stiffness Comparison
    ax3 = fig.add_subplot(gs[1, :2])
    x = np.arange(len(sensor_names))
    width = 0.35
    pred_stiffness = predicted_damage
    true_stiffness = true_damage.cpu().numpy() if isinstance(true_damage, torch.Tensor) else true_damage
    
    bars1 = ax3.bar(x - width/2, true_stiffness, width, label='True Stiffness', 
                    color='#06A77D', alpha=0.8, edgecolor='black')
    bars2 = ax3.bar(x + width/2, pred_stiffness, width, label='Predicted Stiffness',
                    color='#F18F01', alpha=0.8, edgecolor='black')
    ax3.set_ylabel('Stiffness Factor', fontsize=12, fontweight='bold')
    ax3.set_title('Stiffness Comparison: True vs Predicted', fontsize=13, fontweight='bold')
    ax3.set_xticks(x)
    ax3.set_xticklabels(sensor_names, rotation=45, ha='right')
    ax3.legend(fontsize=10)
    ax3.grid(True, alpha=0.3, axis='y')
    ax3.set_ylim(0, 1.1)
    
    for bars in [bars1, bars2]:
        for bar in bars:
            height = bar.get_height()
            ax3.text(bar.get_x() + bar.get_width()/2., height + 0.02,
                    f'{height:.2f}', ha='center', va='bottom', fontsize=9)
    
    # Plot 4: Damage Severity
    ax4 = fig.add_subplot(gs[1, 2])
    true_damage_sev = 1.0 - true_stiffness
    pred_damage_sev = 1.0 - pred_stiffness
    
    bars1 = ax4.bar(x - width/2, true_damage_sev, width, label='True Damage',
                    color='#C73E1D', alpha=0.8, edgecolor='black')
    bars2 = ax4.bar(x + width/2, pred_damage_sev, width, label='Predicted Damage',
                    color='#6A4C93', alpha=0.8, edgecolor='black')
    ax4.axhline(y=0.1, color='orange', linestyle='--', linewidth=2, label='Damage Threshold')
    ax4.set_ylabel('Damage Severity', fontsize=11, fontweight='bold')
    ax4.set_title('Damage Severity', fontsize=12, fontweight='bold')
    ax4.set_xticks(x)
    ax4.set_xticklabels(sensor_names, rotation=45, ha='right')
    ax4.legend(fontsize=9)
    ax4.grid(True, alpha=0.3, axis='y')
    
    # Plot 5: Error Analysis
    ax5 = fig.add_subplot(gs[2, :])
    stiffness_error = np.abs(pred_stiffness - true_stiffness)
    damage_error = np.abs(pred_damage_sev - true_damage_sev)
    
    bars1 = ax5.bar(x - width/2, stiffness_error, width, label='Stiffness Error',
                    color='#2E86AB', alpha=0.8, edgecolor='black')
    bars2 = ax5.bar(x + width/2, damage_error, width, label='Damage Severity Error',
                    color='#A23B72', alpha=0.8, edgecolor='black')
    ax5.set_ylabel('Absolute Error', fontsize=12, fontweight='bold')
    ax5.set_xlabel('Sensors', fontsize=12, fontweight='bold')
    ax5.set_title('Prediction Error Analysis', fontsize=13, fontweight='bold')
    ax5.set_xticks(x)
    ax5.set_xticklabels(sensor_names)
    ax5.legend(fontsize=10)
    ax5.grid(True, alpha=0.3, axis='y')
    
    for bars in [bars1, bars2]:
        for bar in bars:
            height = bar.get_height()
            if height > 0.01:
                ax5.text(bar.get_x() + bar.get_width()/2., height + 0.005,
                        f'{height:.3f}', ha='center', va='bottom', fontsize=8)
    
    plt.suptitle(f'PSO Damage Localization Results - {case_name}', 
                fontsize=16, fontweight='bold', y=0.995)
    plt.savefig(save_path, dpi=300, bbox_inches='tight', facecolor='white')
    plt.close()
    print(f"   ✅ Graph saved: {save_path}")

def plot_overall_performance(all_metrics, scenarios, save_path):
    """Plot overall performance across all scenarios"""
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    scenario_names = [s['name'] for s in scenarios]
    
    # Plot 1: Accuracy across scenarios
    ax = axes[0, 0]
    accuracies = [m['accuracy'] for m in all_metrics]
    colors = ['#06A77D' if acc > 0.8 else '#F18F01' if acc > 0.6 else '#C73E1D' for acc in accuracies]
    ax.bar(range(len(accuracies)), accuracies, color=colors, alpha=0.8, edgecolor='black')
    ax.set_ylabel('Accuracy', fontsize=12, fontweight='bold')
    ax.set_title('Accuracy Across Scenarios', fontsize=13, fontweight='bold')
    ax.set_xticks(range(len(scenario_names)))
    ax.set_xticklabels(scenario_names, rotation=45, ha='right', fontsize=8)
    ax.axhline(y=0.8, color='green', linestyle='--', alpha=0.5, label='Good (0.8)')
    ax.axhline(y=0.6, color='orange', linestyle='--', alpha=0.5, label='Fair (0.6)')
    ax.set_ylim(0, 1.1)
    ax.legend()
    ax.grid(True, alpha=0.3, axis='y')
    
    # Plot 2: All metrics comparison
    ax = axes[0, 1]
    x = np.arange(len(all_metrics))
    width = 0.2
    precisions = [m['precision'] for m in all_metrics]
    recalls = [m['recall'] for m in all_metrics]
    f1_scores = [m['f1_score'] for m in all_metrics]
    
    ax.bar(x - width, accuracies, width, label='Accuracy', color='#06A77D', alpha=0.8)
    ax.bar(x, precisions, width, label='Precision', color='#F18F01', alpha=0.8)
    ax.bar(x + width, recalls, width, label='Recall', color='#C73E1D', alpha=0.8)
    ax.bar(x + 2*width, f1_scores, width, label='F1-Score', color='#6A4C93', alpha=0.8)
    ax.set_ylabel('Score', fontsize=12, fontweight='bold')
    ax.set_title('All Metrics Comparison', fontsize=13, fontweight='bold')
    ax.set_xticks(x + width/2)
    ax.set_xticklabels(range(1, len(all_metrics)+1))
    ax.set_xlabel('Scenario Number', fontsize=11, fontweight='bold')
    ax.legend()
    ax.grid(True, alpha=0.3, axis='y')
    ax.set_ylim(0, 1.1)
    
    # Plot 3: Confusion Matrix
    ax = axes[1, 0]
    total_tp = sum(m['true_positives'] for m in all_metrics)
    total_fp = sum(m['false_positives'] for m in all_metrics)
    total_tn = sum(m['true_negatives'] for m in all_metrics)
    total_fn = sum(m['false_negatives'] for m in all_metrics)
    
    confusion = np.array([[total_tn, total_fp], [total_fn, total_tp]])
    im = ax.imshow(confusion, cmap='Blues', alpha=0.8)
    ax.set_xticks([0, 1])
    ax.set_yticks([0, 1])
    ax.set_xticklabels(['Predicted Healthy', 'Predicted Damaged'], fontsize=10)
    ax.set_yticklabels(['True Healthy', 'True Damaged'], fontsize=10)
    ax.set_title('Cumulative Confusion Matrix', fontsize=13, fontweight='bold')
    
    for i in range(2):
        for j in range(2):
            ax.text(j, i, confusion[i, j], ha="center", va="center", 
                   color="black", fontsize=20, fontweight='bold')
    plt.colorbar(im, ax=ax)
    
    # Plot 4: Distribution
    ax = axes[1, 1]
    metrics_data = [accuracies, precisions, recalls, f1_scores]
    positions = [1, 2, 3, 4]
    ax.boxplot(metrics_data, positions=positions, widths=0.6, patch_artist=True,
               boxprops=dict(facecolor='lightblue', alpha=0.7),
               medianprops=dict(color='red', linewidth=2))
    ax.set_xticklabels(['Accuracy', 'Precision', 'Recall', 'F1-Score'], fontsize=11)
    ax.set_ylabel('Score', fontsize=12, fontweight='bold')
    ax.set_title('Performance Metrics Distribution', fontsize=13, fontweight='bold')
    ax.grid(True, alpha=0.3, axis='y')
    ax.set_ylim(0, 1.1)
    
    means = [np.mean(data) for data in metrics_data]
    ax.plot(positions, means, 'D', color='green', markersize=10, label='Mean', zorder=3)
    ax.legend()
    
    plt.suptitle('PSO Damage Localization - Overall Performance Analysis', 
                fontsize=16, fontweight='bold', y=0.995)
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight', facecolor='white')
    plt.close()
    print(f"   ✅ Overall performance graph: {save_path}")

# ============================================================================
# MAIN PIPELINE
# ============================================================================

def run_pso_simulation(healthy_paths=None, healthy_signals_tensor=None, n_particles=50, 
                      n_iterations=50, n_damage_scenarios=15, device='cuda', output_dir='pso_results'):
    """Run PSO simulation on multiple healthy datasets"""
    os.makedirs(output_dir, exist_ok=True)
    device = torch.device(device if torch.cuda.is_available() else 'cpu')
    print(f"🚀 Using device: {device}")
    
    sensor_names = ['Sensor_24', 'Sensor_31', 'Sensor_32']
    max_length = 1000
    
    # Check if pre-computed tensor is provided
    if healthy_signals_tensor is not None:
        healthy_signals = healthy_signals_tensor.to(device)
        print(f"\n📊 Using pre-computed healthy signals tensor")
        print(f"   📏 Healthy signals shape: {healthy_signals.shape}")
    else:
        # Load and normalize all healthy signals from files
        print(f"\n📊 Loading {len(healthy_paths)} healthy datasets...")
        healthy_signals_list = []
        
        for path in tqdm(healthy_paths):
            signals = load_sensor_data(path, sensor_names)
            if signals is not None:
                normalized_signals = []
                for signal in signals:
                    if len(signal) > max_length:
                        signal = signal[:max_length]
                    elif len(signal) < max_length:
                        signal = np.pad(signal, (0, max_length - len(signal)), mode='constant')
                    normalized_signals.append(signal)
                healthy_signals_list.append(np.array(normalized_signals))
        
        print(f"   ✅ Successfully loaded {len(healthy_signals_list)} healthy datasets")
        
        if len(healthy_signals_list) == 0:
            raise ValueError("No healthy datasets loaded! Please provide valid CSV files or a healthy_signals_tensor.")
        
        # Calculate ensemble baseline
        healthy_signals_array = np.array(healthy_signals_list)
        healthy_baseline = np.mean(healthy_signals_array, axis=0)
        healthy_signals = torch.tensor(healthy_baseline, dtype=torch.float32).to(device)
        print(f"   📏 Healthy signals shape: {healthy_signals.shape}")
    
    # Define damage scenarios
    damage_scenarios = [
        {'name': 'Single_Sensor24_Severe', 'sensors': [0], 'severity': 0.3},
        {'name': 'Single_Sensor24_Moderate', 'sensors': [0], 'severity': 0.6},
        {'name': 'Single_Sensor31_Severe', 'sensors': [1], 'severity': 0.3},
        {'name': 'Single_Sensor31_Moderate', 'sensors': [1], 'severity': 0.6},
        {'name': 'Single_Sensor32_Severe', 'sensors': [2], 'severity': 0.3},
        {'name': 'Single_Sensor32_Moderate', 'sensors': [2], 'severity': 0.6},
        {'name': 'Double_S24_S31_Severe', 'sensors': [0, 1], 'severity': 0.4},
        {'name': 'Double_S24_S31_Moderate', 'sensors': [0, 1], 'severity': 0.7},
        {'name': 'Double_S24_S32_Severe', 'sensors': [0, 2], 'severity': 0.4},
        {'name': 'Double_S31_S32_Moderate', 'sensors': [1, 2], 'severity': 0.7},
        {'name': 'Triple_All_Mild', 'sensors': [0, 1, 2], 'severity': 0.8},
        {'name': 'Triple_All_Moderate', 'sensors': [0, 1, 2], 'severity': 0.6},
        {'name': 'Triple_All_Severe', 'sensors': [0, 1, 2], 'severity': 0.4},
        {'name': 'Single_Sensor24_Mild', 'sensors': [0], 'severity': 0.85},
        {'name': 'Single_Sensor32_Mild', 'sensors': [2], 'severity': 0.85},
    ]
    
    results = []
    all_metrics = []
    
    print(f"\n🔬 Running PSO on {len(damage_scenarios[:n_damage_scenarios])} damage scenarios...\n")
    
    for idx, scenario in enumerate(damage_scenarios[:n_damage_scenarios]):
        print(f"{'='*70}")
        print(f"🔧 Scenario {idx+1}/{min(n_damage_scenarios, len(damage_scenarios))}: {scenario['name']}")
        print(f"{'='*70}")
        
        try:
            damaged_signals, true_damage = create_synthetic_damage(healthy_signals, scenario)
            print(f"   🎯 True damage: {(1 - true_damage.cpu().numpy()).round(3).tolist()}")
            
            pso = BridgePSO(n_particles=n_particles, n_elements=len(sensor_names),
                          bounds=(0.1, 1.0), c1=0.5, c2=0.3, w=0.9, device=device)
            
            best_damage = pso.optimize(healthy_signals, damaged_signals, n_iterations, verbose=True)
            predicted_damage = best_damage.cpu().numpy()
            metrics = evaluate_pso_results(predicted_damage, true_damage.cpu().numpy())
            
            print(f"\n   📊 Metrics: Acc={metrics['accuracy']:.3f}, Prec={metrics['precision']:.3f}, "
                  f"Recall={metrics['recall']:.3f}, F1={metrics['f1_score']:.3f}")
            
            result = {
                'scenario': scenario['name'],
                'true_damage_sensors': scenario['sensors'],
                'true_severity': scenario['severity'],
                'predicted_stiffness': predicted_damage.tolist(),
                'predicted_damage_severity': (1.0 - predicted_damage).tolist(),
                'final_fitness': pso.global_best_score,
                'sensor_names': sensor_names,
                **metrics
            }
            results.append(result)
            all_metrics.append(metrics)
            
            save_path = os.path.join(output_dir, f"pso_scenario_{idx+1:02d}_{scenario['name']}.png")
            plot_pso_results(pso.history, predicted_damage, true_damage, sensor_names, 
                           metrics, scenario['name'], save_path)
            
            print(f"\n   ✨ Predictions:")
            for sensor, pred_stiff, pred_damage in zip(sensor_names, predicted_damage, 1-predicted_damage):
                status = "🔴 DAMAGED" if pred_damage > 0.1 else "🟢 HEALTHY"
                print(f"      {sensor}: Stiffness={pred_stiff:.3f}, Damage={pred_damage:.3f} [{status}]")
            
        except Exception as e:
            print(f"   ❌ Error: {e}")
            continue
    
    # Save results
    print(f"\n{'='*70}")
    print("💾 Saving results...")
    print(f"{'='*70}")
    
    results_df = pd.DataFrame([{
        'scenario': r['scenario'],
        'true_damage_sensors': str(r['true_damage_sensors']),
        'true_severity': r['true_severity'],
        **{f'{sensor}_pred_stiffness': s for sensor, s in zip(sensor_names, r['predicted_stiffness'])},
        **{f'{sensor}_pred_damage': d for sensor, d in zip(sensor_names, r['predicted_damage_severity'])},
        'final_fitness': r['final_fitness'],
        'accuracy': r['accuracy'],
        'precision': r['precision'],
        'recall': r['recall'],
        'f1_score': r['f1_score']
    } for r in results])
    
    csv_path = os.path.join(output_dir, 'pso_damage_localization_results.csv')
    results_df.to_csv(csv_path, index=False)
    print(f"   ✅ Results CSV: {csv_path}")
    
    summary_stats = {
        'total_scenarios': len(results),
        'mean_accuracy': np.mean([m['accuracy'] for m in all_metrics]),
        'mean_precision': np.mean([m['precision'] for m in all_metrics]),
        'mean_recall': np.mean([m['recall'] for m in all_metrics]),
        'mean_f1_score': np.mean([m['f1_score'] for m in all_metrics]),
        'std_accuracy': np.std([m['accuracy'] for m in all_metrics]),
        'std_precision': np.std([m['precision'] for m in all_metrics]),
        'std_recall': np.std([m['recall'] for m in all_metrics]),
        'std_f1_score': np.std([m['f1_score'] for m in all_metrics])
    }
    
    json_path = os.path.join(output_dir, 'pso_summary_statistics.json')
    with open(json_path, 'w') as f:
        json.dump(summary_stats, f, indent=4)
    print(f"   ✅ Summary JSON: {json_path}")
    
    plot_overall_performance(all_metrics, damage_scenarios[:n_damage_scenarios], 
                           os.path.join(output_dir, 'pso_overall_performance.png'))
    
    print(f"\n{'='*70}")
    print("✅ PSO SIMULATION COMPLETE!")
    print(f"{'='*70}")
    print(f"\n📈 Overall Performance:")
    print(f"   Mean Accuracy:  {summary_stats['mean_accuracy']:.4f} ± {summary_stats['std_accuracy']:.4f}")
    print(f"   Mean Precision: {summary_stats['mean_precision']:.4f} ± {summary_stats['std_precision']:.4f}")
    print(f"   Mean Recall:    {summary_stats['mean_recall']:.4f} ± {summary_stats['std_recall']:.4f}")
    print(f"   Mean F1-Score:  {summary_stats['mean_f1_score']:.4f} ± {summary_stats['std_f1_score']:.4f}")
    
    return results, summary_stats

# ============================================================================
# EXECUTION
# ============================================================================

if __name__ == "__main__":
    # Example usage
    print("="*70)
    print("  PSO-BASED BRIDGE DAMAGE LOCALIZATION SYSTEM")
    print("="*70)
    
    # Find all healthy CSV files
    healthy_csv_pattern = "healthy_*.csv"  # Modify this pattern as needed
    healthy_paths = glob(healthy_csv_pattern)
    
    if len(healthy_paths) == 0:
        print("\n⚠️  No healthy CSV files found!")
        print("   Please place healthy sensor data CSV files in the current directory")
        print("   Expected pattern: healthy_*.csv")
        print("\n   Creating synthetic demo instead...")
        
        # Create synthetic demo data
        n_sensors = 3
        n_samples = 1000
        synthetic_signals = []
        
        for i in range(5):  # 5 synthetic healthy datasets
            sensor_data = []
            for s in range(n_sensors):
                # Generate synthetic vibration signals
                t = np.linspace(0, 10, n_samples)
                freq = 2 + s * 0.5
                signal = np.sin(2 * np.pi * freq * t) + np.random.normal(0, 0.1, n_samples)
                sensor_data.append(signal)
            synthetic_signals.append(np.array(sensor_data))
        
        # Convert to tensor
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        healthy_baseline = np.mean(synthetic_signals, axis=0)
        healthy_signals = torch.tensor(healthy_baseline, dtype=torch.float32).to(device)
        
        print(f"\n✅ Created synthetic healthy signals: {healthy_signals.shape}")
        print(f"🚀 Running PSO with synthetic data...\n")
        
        # Run simulation with synthetic data
        results, summary = run_pso_simulation(
            healthy_signals_tensor=healthy_signals,  # Pass the pre-computed tensor
            n_particles=30,
            n_iterations=40,
            n_damage_scenarios=10,
            device='cuda',
            output_dir='pso_results_synthetic'
        )
        
    else:
        print(f"\n✅ Found {len(healthy_paths)} healthy CSV files")
        print("   Files:", [Path(p).name for p in healthy_paths[:5]], "..." if len(healthy_paths) > 5 else "")
        
        # Run simulation with real data
        results, summary = run_pso_simulation(
            healthy_paths=healthy_paths,
            n_particles=50,
            n_iterations=50,
            n_damage_scenarios=15,
            device='cuda',
            output_dir='pso_results'
        )
    
    print("\n" + "="*70)
    print("  SIMULATION COMPLETED SUCCESSFULLY!")
    print("="*70)
    print("\n📁 Check the output directory for:")
    print("   • Individual scenario plots")
    print("   • Overall performance analysis")
    print("   • Results CSV file")
    print("   • Summary statistics JSON")
    print("\n🎉 All done!")